In [1]:
import scanpy as sc
import pandas as pd
import os
from dca.api import dca
import tensorflow as tf
from tensorflow.python.keras.backend import set_session

2021-09-17 14:12:27.613763: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-17 14:12:27.613780: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/users/mscherer/software/anaconda3/envs/dca/lib/python3.8/site-packages/kopt/config.py:60: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  _config = yaml.load(open(_config_path))


### INPUT: Specify which sample you want to analyze. Currently available are:

- Sample2_70_percent
- Sample2_80_percent
- Sample3_default
- Sample4_70_percent
- Sample4_80_percent
- Sample5_70_percent
- Sample5_80_percent

In [2]:
tf.compat.v1.enable_eager_execution()

In [3]:
sample = 'Sample5_80_percent'

### Loading the read count data and potentially removing cell doublets/mulitplets.

In [4]:
#dat = pd.read_table('/users/lvelten/project/Methylome/analysis/missionbio/tapestri/' + sample + '/tsv/' + sample + '.barcode.cell.distribution.tsv',sep='\t')
dat = pd.read_table('/users/lvelten/project/Methylome/analysis/missionbio/downsampling/Sample5_70_percent/tsv/Sample5_70_percent.barcode.cell.distribution.tsv',sep='\t')
#dat = (dat*0.1).round()
remove_mixed = True
if remove_mixed:
    #clust_file = pd.read_csv('/users/lvelten/project/Methylome/analysis/missionbio/tapestri/'+ sample + '/tsv/cluster_assignment.tsv',
    #                        sep='\t')
    clust_file = pd.read_csv('/users/lvelten/project/Methylome/analysis/missionbio/downsampling/Sample5/percent_1/tsv/cluster_assignment.tsv',
                            sep='\t')
    drop_rows = clust_file['Barcode'][clust_file['CellType']=='Mixed']
    dat = dat.drop(drop_rows)

### Creating an AnnData object from the read counts.

In [5]:
adata = sc.AnnData(dat)
sc.pp.filter_genes(adata,min_cells=1)

### Running the autoencoder: Currently, we have the following available architectures:

- 'meth-encoder': A classical autoencoder, where the dispersion parameters of both NB-distributions can be freely selected by the model.
- 'meth-encoder-constant': An autoencoder, where the dispersion parameters is fixed for each gene. This lowers the number of paramters substantially.
- 'meth-encoder-poisson': An autoencoder, which mixes a negative bionomial distribution (foreground) with a Poisson distribution as the background
- 'meth-encoder-poisson-constant': Same as the above, but with a fixed dispersion for each gene
- 'meth-encoder-simple': Instead of having a dense layer representing the mean per cell and amplicon, we use a constant mean per amplicon for both the foreground and the background distribution.

In [6]:
res = dca(adata,
          ae_type='meth-encoder-constant',
          return_info=True,
          return_model=True,
          return_bottleneck=True,
          verbose=True,
          early_stop=50,
          epochs=300,
          network_kwds={'debug': True},
          init='glorot_normal',
          hidden_size=(16,8,16))

dca: Successfully preprocessed 199 genes and 1720 cells.


2021-09-17 14:12:29.819020: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-17 14:12:29.819153: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-17 14:12:29.819163: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-17 14:12:29.819178: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CZC041BTPK): /proc/driver/nvidia/version does not exist


2021-09-17 14:12:30,022 [WARNING] From /users/lvelten/project/Methylome/src/error_mod/dca/dca/train.py:41: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

2021-09-17 14:12:30.024122: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
count (InputLayer)              [(None, 199)]        0                                            
__________________________________________________________________________________________________
enc0 (Dense)                    (None, 16)           3200        count[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16)           48          enc0[0][0]                       
__________________________________________________________________________________________________
enc0_act (Activation)           (None, 16)           0           batch_normalization[0][0]        
____________________________________________________________________________________________

2021-09-17 14:12:30.906059: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2021-09-17 14:12:30.928464: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz


...

/users/mscherer/software/anaconda3/envs/dca/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


1548/1548 [==============================] - 0s 41us/sample - loss: 83.9849 - val_loss: 68.6363
Epoch 4/300
1548/1548 [==============================] - 0s 40us/sample - loss: 66.9919 - val_loss: 51.7414
Epoch 5/300
1548/1548 [==============================] - 0s 41us/sample - loss: 52.1279 - val_loss: 40.1261
Epoch 6/300
1548/1548 [==============================] - 0s 46us/sample - loss: 39.1317 - val_loss: 31.6256
Epoch 7/300
1548/1548 [==============================] - 0s 48us/sample - loss: 28.3987 - val_loss: 24.5974
Epoch 8/300
1548/1548 [==============================] - 0s 47us/sample - loss: 20.2949 - val_loss: 18.5912
Epoch 9/300
1548/1548 [==============================] - 0s 46us/sample - loss: 14.4283 - val_loss: 13.9890
Epoch 10/300
1548/1548 [==============================] - 0s 43us/sample - loss: 10.5431 - val_loss: 10.3315
Epoch 11/300
1548/1548 [==============================] - 0s 44us/sample - loss: 8.4493 - val_loss: 8.2764
Epoch 12/300
1548/1548 [================

1548/1548 [==============================] - 0s 57us/sample - loss: 5.6496 - val_loss: 5.7248
Epoch 80/300
1548/1548 [==============================] - 0s 52us/sample - loss: 5.6503 - val_loss: 5.7246
Epoch 81/300
1548/1548 [==============================] - 0s 52us/sample - loss: 5.6504 - val_loss: 5.7245
Epoch 82/300
1548/1548 [==============================] - 0s 46us/sample - loss: 5.6505 - val_loss: 5.7260
Epoch 83/300
1548/1548 [==============================] - 0s 44us/sample - loss: 5.6506 - val_loss: 5.7243
Epoch 84/300
1548/1548 [==============================] - 0s 41us/sample - loss: 5.6489 - val_loss: 5.7234
Epoch 85/300
1548/1548 [==============================] - 0s 41us/sample - loss: 5.6502 - val_loss: 5.7235
Epoch 86/300
1548/1548 [==============================] - 0s 42us/sample - loss: 5.6491 - val_loss: 5.7229
Epoch 87/300
1548/1548 [==============================] - 0s 41us/sample - loss: 5.6489 - val_loss: 5.7231
Epoch 88/300
1548/1548 [==========================

1548/1548 [==============================] - 0s 40us/sample - loss: 5.6417 - val_loss: 5.7178
Epoch 156/300
1376/1548 [=========================>....] - ETA: 0s - loss: 5.6391
Epoch 00156: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
1548/1548 [==============================] - 0s 66us/sample - loss: 5.6411 - val_loss: 5.7172
Epoch 157/300
1548/1548 [==============================] - 0s 44us/sample - loss: 5.6392 - val_loss: 5.7160
Epoch 158/300
1548/1548 [==============================] - 0s 41us/sample - loss: 5.6400 - val_loss: 5.7157
Epoch 159/300
1548/1548 [==============================] - 0s 41us/sample - loss: 5.6386 - val_loss: 5.7156
Epoch 160/300
1548/1548 [==============================] - 0s 40us/sample - loss: 5.6381 - val_loss: 5.7157
Epoch 161/300
1548/1548 [==============================] - 0s 43us/sample - loss: 5.6403 - val_loss: 5.7156
Epoch 162/300
1548/1548 [==============================] - 0s 42us/sample - loss: 5.6386 - val_loss: 5.7155
E

1548/1548 [==============================] - 0s 44us/sample - loss: 5.6407 - val_loss: 5.7155
Epoch 223/300
1548/1548 [==============================] - 0s 42us/sample - loss: 5.6375 - val_loss: 5.7154
Epoch 224/300
1548/1548 [==============================] - 0s 41us/sample - loss: 5.6378 - val_loss: 5.7154
Epoch 225/300
1548/1548 [==============================] - 0s 41us/sample - loss: 5.6388 - val_loss: 5.7155
Epoch 226/300
1548/1548 [==============================] - 0s 41us/sample - loss: 5.6381 - val_loss: 5.7155
Epoch 227/300
1440/1548 [==========================>...] - ETA: 0s - loss: 5.6382
Epoch 00227: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-10.
1548/1548 [==============================] - 0s 40us/sample - loss: 5.6378 - val_loss: 5.7154
Epoch 228/300
1548/1548 [==============================] - 0s 41us/sample - loss: 5.6378 - val_loss: 5.7154
Epoch 229/300
1548/1548 [==============================] - 0s 42us/sample - loss: 5.6375 - val_loss: 5.7154
Ep

### Writing the data on disk.

In [7]:
#di = '/users/lvelten/project/Methylome/analysis/missionbio/tapestri/' + sample + '/methylation_autoencoder/'
di = '/users/lvelten/project/Methylome/analysis/missionbio/downsampling/Sample5/percent_1/methylation_autoencoder/'
if not os.path.isdir(di):
    os.mkdir(di)
    
pd.DataFrame(adata.obsm['X_meth_value']).to_csv(di + 'mixture_prob_dca.csv')

In [8]:
pd.DataFrame(adata.obsm['X_bottleneck']).to_csv(di + 'bottleneck.csv')
pd.DataFrame(adata.obsm['X_enzyme_activity']).to_csv(di + 'enzyme.csv')
pd.DataFrame(adata.obsm['mean1_norm']).to_csv(di + 'mean1_norm.csv')
pd.DataFrame(adata.obsm['alpha']).to_csv(di + 'alpha.csv')
pd.DataFrame(adata.obs.size_factors).to_csv(di + 'size_factors.csv')

In [9]:
pd.DataFrame(adata.var['meth_dispersion1']).to_csv(di + 'meth_dispersion1.csv')
pd.DataFrame(adata.var['meth_dispersion2']).to_csv(di + 'meth_dispersion2.csv')

In [10]:
adata.obsm['X_enzyme_activity'].min()

0.2264202